In [1]:
#ProstateX final 

#Andrii Trokoz



'''
Importing all the necessary libraries and frameworks in order to start image processing 
'''
import sys
import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
import os
import fnmatch
import shutil
from ipywidgets import interact, fixed
import pandas as pd
import warnings


def myshow(image_arr,slice_n):
    plt.imshow(image_arr[slice_n,:,:],cmap='gray')
    plt.show()

In [3]:
#reading .csv in order to get all of necessary coordinates for cropping 
data_image = pd.read_csv('C://Image Processing/ProstateX-TestLesionInformation/ProstateX-TrainingLesionInformationv2/ProstateX-Images-Train.csv')



In [5]:
#Extracting the names of each image for the further processing from Data Frame, we need this to match names of files with pos coordinates
data_image.loc[:,'match'] = '' #creates new column where we`ll put the matching names 

for i in range(len(data_image)):  #names for further matching with images` names
    data_image.loc[i,'match'] = data_image.loc[i,'DCMSerDescr']+'_' + data_image.loc[i,'ProxID'].replace('-','_')+'.nii.gz'

    
    
data_image.loc[:,('match','Name','ProxID')].head(5)

,match,Name,ProxID
0,ep2d_diff_tra_DYNDIST_ADC_ProstateX_0000.nii.gz,ep2d_diff_tra_DYNDIST_ADC0,ProstateX-0000
1,ep2d_diff_tra_DYNDIST_ProstateX_0000.nii.gz,ep2d_diff_tra_DYNDIST0,ProstateX-0000
2,ep2d_diff_tra_DYNDIST_ProstateX_0000.nii.gz,ep2d_diff_tra_DYNDIST1,ProstateX-0000
3,ep2d_diff_tra_DYNDIST_ProstateX_0000.nii.gz,ep2d_diff_tra_DYNDIST2,ProstateX-0000
4,ep2d_diff_tra_DYNDISTCALC_BVAL_ProstateX_0000....,ep2d_diff_tra_DYNDISTCALC_BVAL0,ProstateX-0000


In [6]:
'''
Creating separate DataFrames for each of modalites. 

It is going to be: t2_df, adc_df, bval_df and ktrans_df

'''



#creating lists for further processing, renaming and removing the duplicates in the directory  
t2_dir=os.listdir('C://Image Processing/ProstateX/resampled/')
t2_match = [i for i in t2_dir if 't2' in i ]

def make_match_list(directory, matching_value): 
    #generating a list of matching names
    match=[i for i in directory if matching_value in i]
    for i in range(len(match)): 
        if match[i].split('_')[-1][0] !='0':
            match[i]=match[i][:-12] + '.nii.gz'
    
    match_list=[]
    for i in match: 
        if i not in match_list:
            match_list.append(i)
            
    return match_list

#macth lists
t2_list = make_match_list(t2_dir,'t2')

adc_list = make_match_list(t2_dir,'ADC')

bval_list = make_match_list(t2_dir, 'BVAL')



#matching dataframes

#Creating a DataFrame with names of files 

t2_df = data_image[data_image['match'].isin(t2_list)]
t2_df.index = range(len(t2_df))

adc_df = data_image[data_image['match'].isin(adc_list)]
adc_df.index = range(len(adc_df))

bval_df = data_image[data_image['match'].isin(bval_list)]
bval_df.index = range(len(bval_df))

In [9]:
#we have some 
t2_df.loc[:,('match', 'DCMSerNum')]

try:
    for i in range(len(t2_df)):
        
        if t2_df.loc[i,'match']==t2_df.loc[i+1,'match'] and t2_df.loc[i,'DCMSerNum'] != t2_df.loc[i+1,'DCMSerNum']:
            
            if len(str(t2_df.loc[i,'DCMSerNum'])) == 1:
                while len(str(t2_df.loc[i+1,'DCMSerNum'])) != 2:
                    t2_df.loc[i,'match'] = t2_df.loc[i,'match'].strip('.nii.gz')+'_s00'+str(t2_df.loc[i,'DCMSerNum'])+'.nii.gz'
                    t2_df.loc[i+1,'match'] = t2_df.loc[i+1,'match'].strip('.nii.gz')+'_s00'+str(t2_df.loc[i+1,'DCMSerNum'])+'.nii.gz'
                else: 
                    t2_df.loc[i+1,'match'] = t2_df.loc[i,'match'].strip('.nii.gz')+'_s0'+str(t2_df.loc[i,'DCMSerNum'])+'.nii.gz'
            
            elif len(str(t2_df.loc[i,'DCMSerNum'])) == 2:
                while len(str(t2_df.loc[i+1,'DCMSerNum'])) != 2:
                    t2_df.loc[i,'match'] = t2_df.loc[i,'match'].strip('.nii.gz')+'_s0'+str(t2_df.loc[i,'DCMSerNum'])+'.nii.gz'
                    t2_df.loc[i+1,'match'] = t2_df.loc[i+1,'match'].strip('.nii.gz')+'_s00'+str(t2_df.loc[i+1,'DCMSerNum'])+'.nii.gz'
                else:
                    t2_df.loc[i+1,'match'] = t2_df.loc[i,'match'].strip('.nii.gz')+'_s0'+str(t2_df.loc[i,'DCMSerNum'])+'.nii.gz'
except: KeyError



C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
for i in range(len(t2_df)):
    if t2_df.loc[i,'match']==t2_df.loc[i+1,'match'] and t2_df.loc[i,'DCMSerNum'] != t2_df.loc[i+1,'DCMSerNum']:
        print(type(str(t2_df.loc[i,'DCMSerNum'])))


In [13]:
t2_df.loc[t2_df['DCMSerNum']==10]

,ProxID,Name,fid,pos,WorldMatrix,ijk,TopLevel,SpacingBetweenSlices,VoxelSpacing,Dim,DCMSerDescr,DCMSerNum,match
2,ProstateX-0001,t2_tse_tra0,1,-40.5367071921656 29.320722668457 -16.70766907...,"0.5,1.02552e-010,-4.64581e-017,-119.205,-1.019...",157 186 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,10,t2_tse_tra_ProstateX_0001_s0010.nii.gz
224,ProstateX-0123,t2_tse_tra0,1,20.8643 30.9437 29.9455,"0.5,1.02552e-010,1.40962e-016,-83.3017,-1.0089...",208 227 7,NaN,3.0,"0.5,0.5,3",384x384x21x1,t2_tse_tra,10,t2_tse_tra_ProstateX_0123_s0010.nii.gz
377,ProstateX-0186,t2_tse_tra0,2,11.8551 32.938 20.8921,"0.5,1.02552e-010,1.83796e-017,-92.2174,-9.6119...",208 217 7,NaN,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,10,t2_tse_tra_ProstateX_0186_s0010.nii.gz
378,ProstateX-0186,t2_tse_tra0,1,-0.209802 19.6911 22.6179,"0.5,1.02552e-010,1.83796e-017,-92.2174,-9.6119...",184 191 6,NaN,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,10,t2_tse_tra_ProstateX_0186.nii.gz


In [ ]:
matching_dict = {i: k for i in data_all['ProxID'] for k in directory if i.split('-')[-1] in k}

matching_dict

In [ ]:
#Making new column with physical coordinates of the findings 
#In order to do that, we should first read image, than get pos and finally transfer physical point to index

#first of all we should match ProxID and read image 

#for reading image we have to undestand which image to read, get its path and match it with correct pos

#find ProxID -- find filename -- match proxid with filename -- find pos -- read image -- transform image pos to index

def phys_to_index_col(data, folder):
    directory = os.listdir(folder)
    for i in directory: 
        for k in data['ProxID']:
            if k.split('-')[-1] in i: 
                pos=[float(i) for i in (data['pos'].loc[data['ProxID']==k][0]).split(' ')]
                im = sitk.ReadImage(folder+i)
                data['index'].loc[data['ProxID']==k]=im.TransformPhysicalPointToIndex(pos)



In [ ]:
a=[float(i) for i in (data_all['pos'].loc[data['ProxID']=='ProstateX-0000'][0]).split(' ')]

data_new['ProxID'][1]
data_new.loc[data_new['ProxID']=='ProstateX-0002'].index[0]

a

In [ ]:
data_all['pos'].loc[data['ProxID']=='ProstateX-0191']
directory

In [ ]:
for i in directory:
    for k in data_all['ProxID']:
        if k.split('-')[-1] in i: 
            print(k)
            pos=[float(i) for i in (data_all['pos'].loc[data_all['ProxID']==k][data_all.loc[data['ProxID']==k].index[0]]).split(' ')]
            print(pos)


In [ ]:
extract = sitk.ExtractImageFilter()
extract.SetSize([64,64,0])
extract.SetIndex([153-20, 166-32, 12])
extracted = extract.Execute(im)

extr_arr = sitk.GetArrayFromImage(extracted)
plt.imshow(extr_arr,cmap='gray')

In [ ]:
extract = sitk.ExtractImageFilter()
extract.SetSize([64,64,0])
extract.SetIndex([203-20, 168-32, 9])
extracted = extract.Execute(im)

extr_arr = sitk.GetArrayFromImage(extracted)
plt.imshow(extr_arr,cmap='gray')



In [ ]:
im = sitk.ReadImage('C://Image Processing/ProstateX/t2/t2_tse_tra_ProstateX_0002.nii')
im_arr = sitk.GetArrayFromImage(im)
im_sl_9 = im_arr[9,:,:]
im_sl_9 = sitk.GetImageFromArray(im_sl_9)

extract = sitk.ExtractImageFilter()
extract.SetSize([32,32])
extract.SetIndex([203-20, 168-32])
extracted_image = extract.Execute(im_sl_9)


extrcated_arr = sitk.GetArrayFromImage(extracted_image)

plt.imshow(extrcated_arr, cmap = 'gray')

In [ ]:
myshow(im_arr,9)

In [ ]:
pos = [-27.0102, 41.5467, -26.0469]
ijk = im.TransformPhysicalPointToIndex(pos)

print(ijk)

In [ ]:
im = sitk.ReadImage('C://Image Processing/ProstateX/resampled/ep2d_diff_tra_DYNDIST_ADC_ProstateX_0000.nii')
im_arr = sitk.GetArrayFromImage(im)

im_sl_8 = im_arr[9,:,:]
im_sl_8 = sitk.GetImageFromArray(im_sl_8)

extract = sitk.ExtractImageFilter()
extract.SetSize([36,36])
extract.SetIndex([36, 72 ])
extracted_image = extract.Execute(im_sl_8)


extrcated_arr = sitk.GetArrayFromImage(extracted_image)

plt.imshow(extrcated_arr, cmap = 'gray')

In [ ]:
myshow(im_arr,9)

In [ ]:
data_all.loc[data_all['ProxID']=='ProstateX-0002']